# Tensor Operations

In this tutorial, we'll go over what you can do with tensors. Each supported operation will be written out as a function. Then, all of them will be compiled in a loop and executed with a random input to demonstrate their semantics.

### Imports

In [1]:
import concrete.numpy as hnp
import inspect
import numpy as np

### Inputset Definition

In [2]:
inputset = [np.random.randint(3, 11, size=(3, 2), dtype=np.uint8) for _ in range(10)]

### Supported Operation Definitions

In [3]:
def reshape(x):
    return x.reshape((2, 3))

In [4]:
def flatten(x):
    return x.flatten()

In [5]:
def index(x):
    return x[2, 0]

In [6]:
def slice_(x):
    return x.flatten()[1:5]

In [7]:
def add_scalar(x):
    return x + 10

In [8]:
def add_tensor(x):
    return x + np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8)

In [9]:
def add_tensor_broadcasted(x):
    return x + np.array([1, 10], dtype=np.uint8)

In [10]:
def sub_scalar(x):
    return x + (-1)

In [11]:
def sub_tensor(x):
    return x + (-np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8))

In [12]:
def sub_tensor_broadcasted(x):
    return x + (-np.array([3, 0], dtype=np.uint8))

In [13]:
def mul_scalar(x):
    return x * 2

In [14]:
def mul_tensor(x):
    return x * np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8)

In [15]:
def mul_tensor_broadcasted(x):
    return x * np.array([2, 3], dtype=np.uint8)

In [16]:
def power(x):
    return x ** 2

In [17]:
def truediv(x):
    return x // 2

In [18]:
def dot(x):
    return x.flatten() @ np.array([1, 1, 1, 2, 1, 1], dtype=np.uint8)

In [19]:
def matmul(x):
    return x @ np.array([[1, 2, 3], [3, 2, 1]], dtype=np.uint8)

In [20]:
def clip(x):
    return x.clip(6, 11)

In [21]:
def comparison(x):
    return x > np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8)

In [22]:
def minimum(x):
    return np.minimum(x, np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8))

In [23]:
def maximum(x):
    return np.maximum(x, np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8))

Other than these, we support a lot of numpy functions which you can find more about at [Numpy Support](../howto/numpy_support.md).

### Compilation and Homomorphic Evaluation of Supported Operations

Note that some operations require programmable bootstrapping to work and programmable bootstrapping has a certain probability of failure. Usually, it has more than a 99% probability of success but with big bit-widths, this probability can drop to 95%.

In [24]:
functions = [
    reshape,
    flatten,
    index,
    slice_,
    add_scalar,
    add_tensor,
    add_tensor_broadcasted,
    sub_scalar,
    sub_tensor,
    sub_tensor_broadcasted,
    mul_scalar,
    mul_tensor,
    mul_tensor_broadcasted,
    power,
    truediv,
    dot,
    matmul,
    clip,
    comparison,
    maximum,
    minimum,
]

for function in functions:
    compiler = hnp.NPFHECompiler(function, {"x": "encrypted"})
    circuit = compiler.compile_on_inputset(inputset)
    
    sample = np.random.randint(3, 11, size=(3, 2), dtype=np.uint8)
    result = circuit.run(sample)
    
    print("#######################################################################################")
    
    print()
    print(f"{inspect.getsource(function)}")
    print(f"{function.__name__}({sample.tolist()}) homomorphically evaluates to {result if isinstance(result, int) else result.tolist()}")
    print()

    expected = function(sample)
    if not np.array_equal(result, expected):
        print(f"(It should have been evaluated to {expected if isinstance(expected, int) else expected.tolist()} but it didn't due to an error during PBS)")
        print()

#######################################################################################

def reshape(x):
    return x.reshape((2, 3))

reshape([[7, 10], [9, 10], [7, 9]]) homomorphically evaluates to [[7, 10, 9], [10, 7, 9]]

#######################################################################################

def flatten(x):
    return x.flatten()

flatten([[4, 7], [6, 3], [9, 8]]) homomorphically evaluates to [4, 7, 6, 3, 9, 8]

#######################################################################################

def index(x):
    return x[2, 0]

index([[8, 4], [9, 6], [9, 10]]) homomorphically evaluates to 9

#######################################################################################

def slice_(x):
    return x.flatten()[1:5]

slice_([[5, 9], [7, 4], [7, 3]]) homomorphically evaluates to [9, 7, 4, 7]

#######################################################################################

def add_scalar(x):
    return x + 10

add_scalar([[10, 9], [9, 9], [8, 9]]) 